 Thanks to previous Notebooks 
 https://www.kaggle.com/prashantchandel1097/ensemble-of-multiple-models-lb0-733
 https://www.kaggle.com/prashantchandel1097/ensemble-of-multiple-models-lb0-733
 https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700  
 https://www.kaggle.com/muhammad4hmed/b3-tfidf-knn-boom-p  
 https://www.kaggle.com/parthdhameliya77/pytorch-eca-nfnet-l0-image-tfidf-inference
 

This approach uses following:

* RAPIDs KNN Search using TFIDF for Text and EfficientNet+NFNet Ensemble with ArcFace for Images - (copied from public kernel) 
* Product quantity detection and filtering, in order to differentiate between same product name and different size/quantity.
* Initial cleaning of text by replacing Strings.
* Using Prefix-Tree data structure to find and join words which are sometimes broken in two parts and sometimes not.
* Making all similar mispelled big words same with first and last letter same and max 1 letter difference.
* Adding all posts with same title or phash.
* Filtering text matches using Jaccard Index.

In [ ]:
!pip install ../input/datrie-python-prefix-tree/datrie-0.8.2-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
!pip install ../input/efficientnetkerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/efficientnetkerasapplications/efficientnet-1.1.1-py3-none-any.whl
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import numpy as np
import pandas as pd
import string
import datrie
import gc
import matplotlib.pyplot as plt
import cudf
import cuml
import cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml import PCA
from cuml.neighbors import NearestNeighbors
import tensorflow as tf
import efficientnet.tfkeras as efn
import math
from shutil import copyfile
copyfile(src = "../input/bert-and-tokenization/tokenization.py", dst = "../working/tokenization.py")
import tokenization
import tensorflow_hub as hub

import os
import cv2
import random
from tqdm import tqdm

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import torch
import timm
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
import fasttext as ft

In [ ]:
fullWordsSet = [] 
trie = [] 
jointReplacements = []
fuzzRepDct = []

def stringReplacer(st):
    st = st.replace('(','').replace(')','')
    st = st.replace('[','').replace(']','')
    st = st.replace('{','').replace('}','')
    st = st.replace('!','')
    st = st.replace('#',' ')
    st = st.replace('-',' ')
    st = st.replace(',',' ')
    st = st.replace('\\',' ')
    st = st.replace('/',' ')
    st = st.replace('|',' ')
    st = st.replace(' THE ',' ')
    st = st.replace('"',' ')
    st = st.replace(' X ',' ')
    st = st.replace('=',' ')
    
    if len(st)>0:
        if st[-1]=='.':
            st = st[:-1]
    
    ########
    """
    allSWs = ['ORIGINAL','&','1','EXTRA','PACKING','IMPORT','+','FREE','COD','SELLER','FASHION','SIZE','COLOR','NEW','FULL','FRESH','STOCK','PACK','BEST','REAL','MEMBERITAHU','WAS','MAYROON','EXTRA','BY','NOON AY','BE','MAY','APA','NYA','KAMI','KUNG PAANO','TEM','MEMILIKI','ONE','WHICH','KEPADA','O QUE','IT','TEMPO','ISANG','ILANG','TO','ARE','É','THIS','DE','OR','JIKA','SA','NÓS','UMA','MAAARI','DAN','BAGAIMANA','SÃO','PAMANAHAN','ON','AKO','PACKING','THAT','SAID','QUE','SE','SALITA','NG','AS','WITH','WERE','TIDAK','SEBUAH','FOR','PALAVRA','WE','IBANG','DO','KANYANG','AKAN','&','KUNG','DARIPADA','POR','UNTUK','FROM','MASING-MASING','THE','INI','MENJADI','SER','NGUNIT','DALAM','KUNG ANO','SAYA','HOW','NA','BAWAT','SEBAGAI','PACK','FASHION','CAN','DIZER','ANG','AY','MEMPUNYAI','ITU','GUMAGANA','MAS','OTHER','CADA','DOES','ATAU','TEVE','SABIHIN','ESTE','COLOR','SETIAP','SEORANG','OUTRO','SIYA','LAINNYA','MAGING','YANG','HAD','ORIGINAL','MY','SELLER','YOU','A PARTIR DE','IF','WHAT','DISSE','MELAKUKAN','PADA','OF','THEIR','COM','SOME','BILANG','DARI','KUNG SAAN','O','ANDA','AT','SINABI','QUENTE','VOCÊ','DI','KATA','TAPI','FAZ','LATA','FAZER','FOI','LAIN','PARA','COMO','ORAS','1','ANG KANILANG','OLEH','IMPORT','ITO','THEY','PARA SA','BAHWA','BEST','HAVE','SATU','STOCK','HE','IS','FORAM','SA PAMAMAGITAN NG','BERKATA','NEW','GAWIN',"IT'S",'PANAS','OU','NAGKAROON','DENGAN','KELUAR','MULA SA','E','NAMIN','ADALAH','MEREKA','HOT','FRESH','SEU','BEBERAPA','IA','A','TIME','AND','BUT','WILL','BOLEH','MASA','FULL','OUT','SILA','EACH','PERKATAAN','FORA','ISA','HIS','FREE','+','SIZE','I','REAL','ELE','MAINIT','AN','TELL','BISA','TETAPI','DIA','WAKTU','VONTADE','SA IYO','ELES','ALGUNS','EM','BELIAU','WORD','COD']
    for sw in allSWs:
        st = st.replace(' '+sw+' ',' ')
    """
    #########
    st = st.replace('    ','   ')
    st = st.replace('   ','  ')
    st = st.replace('  ',' ')
    return st

def cleanSlashX(st):
    st2 = ''
    i=0

    while i<len(st):
        if st[i:i+2]=='\X':
            i+=4
        else:
            st2 += st[i]
            i+=1
    return st2

def cleanup_text(text):
    text = cleanSlashX(text)
    text = stringReplacer(text)
    text = "".join([c if ord(c) < 128 else "" for c in text]).strip() 
    return text


def stringReplacer_Joined(st):
    global jointReplacements
    for k,v in jointReplacements.items():
        if k in st:
            st = st.replace(k,v)

    return st

def stringReplacer_Fuzzy(st):
    global fuzzRepDct
    for k,v in fuzzRepDct.items():
        if k in st:
            st = st.replace(k,v)

    return st

In [ ]:
def tokenizeTitle(ttl):
    tl = ttl.split(' ')
    tl = [wd for wd in tl if len(wd)>0]
    #return set(tl[:30])
    return set(tl)

def read_dataset():
    global fullWordsSet, trie, jointReplacements, fuzzRepDct
    
    df = pd.read_csv('../input/shopee-product-matching/test.csv')
    
    df['title'] = df['title'].str.upper()
    df['title'] = df['title'].apply(cleanup_text)
    df['tokens'] = df['title'].apply(tokenizeTitle)

    fullWordsSet = set()
    for i in tqdm(range(df.shape[0])):
        for wrd in df['tokens'].iloc[i]:
            if len(wrd) > 2:
                if not any(char.isdigit() for char in wrd):
                    fullWordsSet.add(wrd)
    fullWordsSet = list(fullWordsSet)
    
    trie = datrie.Trie(string.ascii_uppercase)
    for i,wrd in tqdm(enumerate(fullWordsSet)):
        trie[wrd] = i
        
    jointCnt = 0
    jointReplacements = {}
    for i,wrd in tqdm(enumerate(fullWordsSet)):
        prefxs = trie.prefixes(wrd)
        for j,wrd2 in enumerate(prefxs):
            if wrd2 != wrd:
                wrd3 = wrd[ len(wrd2): ]
                if wrd3 in trie:
                    #print(wrd2,' + ',wrd3,' = ',wrd)
                    #print('*******')
                    jointCnt += 1
                    jointReplacements[ wrd2+' '+wrd3 ] = wrd
    
    
    df['title'] = df['title'].apply(stringReplacer_Joined)
    
    df['tokens'] = df['title'].apply(tokenizeTitle)
    
    fullWordsSet = set()
    for i in tqdm(range(df.shape[0])):
        for wrd in df['tokens'].iloc[i]:
            if len(wrd) > 2:
                if not any(char.isdigit() for char in wrd):
                    fullWordsSet.add(wrd)
    fullWordsSet = list(fullWordsSet)
    
    azDict = {}

    for wrd in tqdm(fullWordsSet):
        if len(wrd) > 1:
            ak = wrd[0]
            zk = wrd[-1]
            azk = ak+zk
            if azk not in azDict:
                azDict[azk] = []
            else:
                azDict[azk].append(wrd)
                
    #1,6 1,7 2,7 2,8 1,8
    mmMax = 1
    minLen = 6
    fuzzRepDct = {}

    for k,v in tqdm(azDict.items()):
        L = v
        used = []
        grps = []
        for w in L:
            if w not in used:
                grp = []
                for w2 in L:
                    if len(w)>=minLen and len(w)==len(w2):
                        mm = 0
                        for i,c in enumerate(w):
                            if c != w2[i]:
                                mm += 1
                        if mm<=mmMax:
                            grp.append(w2)
                            used.append(w2)
                if len(grp)>1:
                    grps.append(grp)   
                    pvt = grp[0]
                    for n in range(1,len(grp)):
                        fuzzRepDct[ grp[n] ] = pvt
                        
    df['title'] = df['title'].apply(stringReplacer_Fuzzy)
    
    del df['tokens']
    
    df_cu = cudf.DataFrame(df)
    image_paths = '../input/shopee-product-matching/test_images/' + df['image']
    return df, df_cu, image_paths

def read_datasetTrain():
    global fullWordsSet, trie, jointReplacements, fuzzRepDct
    df = pd.read_csv('../input/shopee-product-matching/train.csv')
    
    df['title'] = df['title'].str.upper()
    df['title'] = df['title'].apply(cleanup_text)
    df['tokens'] = df['title'].apply(tokenizeTitle)
    
    fullWordsSet = set()
    for i in tqdm(range(df.shape[0])):
        for wrd in df['tokens'].iloc[i]:
            if len(wrd) > 2:
                if not any(char.isdigit() for char in wrd):
                    fullWordsSet.add(wrd)
    fullWordsSet = list(fullWordsSet)
    
    trie = datrie.Trie(string.ascii_uppercase)
    for i,wrd in tqdm(enumerate(fullWordsSet)):
        trie[wrd] = i
        
    jointCnt = 0
    jointReplacements = {}
    for i,wrd in tqdm(enumerate(fullWordsSet)):
        prefxs = trie.prefixes(wrd)
        for j,wrd2 in enumerate(prefxs):
            if wrd2 != wrd:
                wrd3 = wrd[ len(wrd2): ]
                if wrd3 in trie:
                    #print(wrd2,' + ',wrd3,' = ',wrd)
                    #print('*******')
                    jointCnt += 1
                    jointReplacements[ wrd2+' '+wrd3 ] = wrd
    
    
    df['title'] = df['title'].apply(stringReplacer_Joined)
    
    
    df['tokens'] = df['title'].apply(tokenizeTitle)
    
    fullWordsSet = set()
    for i in tqdm(range(df.shape[0])):
        for wrd in df['tokens'].iloc[i]:
            if len(wrd) > 2:
                if not any(char.isdigit() for char in wrd):
                    fullWordsSet.add(wrd)
    fullWordsSet = list(fullWordsSet)
    
    azDict = {}

    for wrd in tqdm(fullWordsSet):
        if len(wrd) > 1:
            ak = wrd[0]
            zk = wrd[-1]
            azk = ak+zk
            if azk not in azDict:
                azDict[azk] = []
            else:
                azDict[azk].append(wrd)
                
    #1,6 1,7 2,7 2,8 1,8
    mmMax = 1
    minLen = 8
    fuzzRepDct = {}

    for k,v in tqdm(azDict.items()):
        L = v
        used = []
        grps = []
        for w in L:
            if w not in used:
                grp = []
                for w2 in L:
                    if len(w)>=minLen and len(w)==len(w2):
                        mm = 0
                        for i,c in enumerate(w):
                            if c != w2[i]:
                                mm += 1
                        if mm<=mmMax:
                            grp.append(w2)
                            used.append(w2)
                if len(grp)>1:
                    grps.append(grp)   
                    pvt = grp[0]
                    for n in range(1,len(grp)):
                        fuzzRepDct[ grp[n] ] = pvt
                        
    df['title'] = df['title'].apply(stringReplacer_Fuzzy)
    
    del df['tokens']
    
    df_cu = cudf.DataFrame(df)
    image_paths = '../input/shopee-product-matching/train_images/' + df['image']
    return df, df_cu, image_paths

In [ ]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
BATCH_SIZE = 8
IMAGE_SIZE = [512, 512]
# Seed
SEED = 42
# Verbosity
VERBOSE = 1
# Number of classes
N_CLASSES = 11011

In [ ]:
# RESTRICT TENSORFLOW TO 2GB OF GPU RAM
# SO THAT WE HAVE 14GB RAM FOR RAPIDS
LIMIT = 2.0
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
print('We will restrict TensorFlow to max %iGB GPU RAM'%LIMIT)
print('then RAPIDS can use %iGB GPU RAM'%(16-LIMIT))

In [ ]:
GET_CV = False
# Flag to check ram allocations (debug)
CHECK_SUB = False

df = cudf.read_csv('../input/shopee-product-matching/test.csv')
# If we are comitting, replace train set for test set and dont get cv
if len(df) > 3:
    GET_CV = False
del df

# Function to get our f1 score
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1


# Function to combine predictions
def combine_predictions(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions'], row['oof_text'], row['oof_hash']])
    return ' '.join( np.unique(x) )

"""
# Function to read out dataset
def read_dataset():
    if GET_CV:
        df = pd.read_csv('../input/shopee-product-matching/train.csv')
        tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
        df['matches'] = df['label_group'].map(tmp)
        df['matches'] = df['matches'].apply(lambda x: ' '.join(x))
        if CHECK_SUB:
            df = pd.concat([df, df], axis = 0)
            df.reset_index(drop = True, inplace = True)
        df_cu = cudf.DataFrame(df)
        image_paths = '../input/shopee-product-matching/train_images/' + df['image']
    else:
        df = pd.read_csv('../input/shopee-product-matching/test.csv')
        df_cu = cudf.DataFrame(df)
        image_paths = '../input/shopee-product-matching/test_images/' + df['image']
        
    return df, df_cu, image_paths

"""
# Function to decode our images
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Function to read our test image and return image
def read_image(image):
    image = tf.io.read_file(image)
    image = decode_image(image)
    return image

# Function to get our dataset that read images
def get_dataset(image):
    dataset = tf.data.Dataset.from_tensor_slices(image)
    dataset = dataset.map(read_image, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [ ]:
# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)
    
    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output
    
    
    

In [ ]:
# Function to get the embeddings of our images with the fine-tuned model
def get_image_embeddings(image_paths):
    embeds = []
    
    margin = ArcMarginProduct(
            n_classes = N_CLASSES, 
            s = 30, 
            m = 0.5, 
            name='head/arc_margin', 
            dtype='float32'
            )

    inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')
    x = efn.EfficientNetB3(weights = None, include_top = False)(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = margin([x, label])
        
    output = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    model.load_weights('../input/efficientnet/EfficientNetB3_512_42.h5')
    model = tf.keras.models.Model(inputs = model.input[0], outputs = model.layers[-4].output)
    chunk = 5000
    iterator = np.arange(np.ceil(len(df) / chunk))
    for j in iterator:
        a = int(j * chunk)
        b = int((j + 1) * chunk)
        image_dataset = get_dataset(image_paths[a:b])
        image_embeddings = model.predict(image_dataset)
        embeds.append(image_embeddings)
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

# Return tokens, masks and segments from a text array or series
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
# Function to get our text title embeddings using a pre-trained bert model
def get_text_embeddings(df, max_len = 70):
    embeds = []
    module_url = "../input/bert-en-uncased-l24-h1024-a16-1"
    bert_layer = hub.KerasLayer(module_url, trainable = True)
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
    text = bert_encode(df['title'].values, tokenizer, max_len = max_len)
    
    margin = ArcMarginProduct(
            n_classes = 11014, 
            s = 30, 
            m = 0.5, 
            name='head/arc_margin', 
            dtype='float32'
            )
    input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    label = tf.keras.layers.Input(shape = (), name = 'label')

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    x = margin([clf_output, label])
    output = tf.keras.layers.Softmax(dtype='float32')(x)
    model = tf.keras.models.Model(inputs = [input_word_ids, input_mask, segment_ids, label], outputs = [output])
    
    model.load_weights('../input/bert-and-tokenization/Bert_123.h5')
    model = tf.keras.models.Model(inputs = model.input[0:3], outputs = model.layers[-4].output)
    chunk = 5000
    iterator = np.arange(np.ceil(len(df) / chunk))
    for j in iterator:
        a = int(j * chunk)
        b = int((j + 1) * chunk)
        text_chunk = ((text[0][a:b], text[1][a:b], text[2][a:b]))
        text_embeddings = model.predict(text_chunk, batch_size = BATCH_SIZE)
        embeds.append(text_embeddings)
    del model
    text_embeddings = np.concatenate(embeds)
    print(f'Our text embeddings shape is {text_embeddings.shape}')
    del embeds
    gc.collect()
    return text_embeddings

In [ ]:
# Function to get 50 nearest neighbors of each image and apply a distance threshold to maximize cv
def get_neighbors(df, embeddings, KNN = 50, image = True):
    
    if len(df) <= 3:
        KNN = 3
    
    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    # Iterate through different thresholds to maximize cv, run this in interactive mode, then replace else clause with a solid threshold
    if GET_CV:
        if image:
            thresholds = list(np.arange(4.5, 5.0, 0.1))
        else:
            thresholds = list(np.arange(31, 35, 1))
        scores = []
        for threshold in thresholds:
            predictions = []
            for k in range(embeddings.shape[0]):
                idx = np.where(distances[k,] < threshold)[0]
                ids = indices[k,idx]
                posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
                predictions.append(posting_ids)
            df['pred_matches'] = predictions
            df['f1'] = f1_score(df['matches'], df['pred_matches'])
            score = df['f1'].mean()
            print(f'Our f1 score for threshold {threshold} is {score}')
            scores.append(score)
        thresholds_scores = pd.DataFrame({'thresholds': thresholds, 'scores': scores})
        max_score = thresholds_scores[thresholds_scores['scores'] == thresholds_scores['scores'].max()]
        best_threshold = max_score['thresholds'].values[0]
        best_score = max_score['scores'].values[0]
        print(f'Our best score is {best_score} and has a threshold {best_threshold}')
        
        # Use threshold
        predictions = []
        for k in range(embeddings.shape[0]):
            ids = np.array([])
            # Because we are predicting the test set that have 70K images and different label groups, confidence should be smaller
            if image:
                idx = np.where(distances[k,] < 3.3)[0]
                ids = indices[k,idx]
            else:
                idx = np.where(distances[k,] < 20.0)[0]
                ids = indices[k,idx]
                if (len(idx)>1):
                    arr = distances[k,np.where(distances[k,]<20)[0]][1:]
                    mean = np.mean(arr)
                    standard_deviation = np.std(arr)
                    if(standard_deviation>0):
                        distance_from_mean = abs(arr - mean)
                        max_deviations = 2
                        not_outlier = distance_from_mean < max_deviations * standard_deviation
                        max_dist = arr[not_outlier][-1]
                        idx = np.where(distances[k,] <= max_dist)[0]
                        ids = indices[k,idx]
            posting_ids = df['posting_id'].iloc[ids].values
            predictions.append(posting_ids)
    
    # Because we are predicting the test set that have 70K images and different label groups, confidence should be smaller
    else:
        predictions = []
        for k in tqdm(range(embeddings.shape[0])):
            ids = np.array([])
            if image:
                idx = np.where(distances[k,] < 3.3)[0]
                ids = indices[k,idx]
            else:
                idx = np.where(distances[k,] < 16.0)[0]
                ids = indices[k,idx]
                if (len(idx)>1):
                    arr = distances[k,np.where(distances[k,]<16.0)[0]][1:]
                    mean = np.mean(arr)
                    standard_deviation = np.std(arr)
                    if(standard_deviation>0):
                        distance_from_mean = abs(arr - mean)
                        max_deviations = 2
                        not_outlier = distance_from_mean < max_deviations * standard_deviation
                        max_dist = arr[not_outlier][-1]
                        idx = np.where(distances[k,] <= max_dist)[0]
                        ids = indices[k,idx]
            
            posting_ids = df['posting_id'].iloc[ids].values
            predictions.append(posting_ids)
        
    del model, distances, indices
    gc.collect()
    return df, predictions

In [ ]:
#df,df_cu,image_paths = read_datasetTrain()
df,df_cu,image_paths = read_dataset()
df.head()

In [ ]:
image_embeddings = get_image_embeddings(image_paths)


In [ ]:
text_embeddings = get_text_embeddings(df)
gc.collect()

In [ ]:
df, image_predictions = get_neighbors(df, image_embeddings, KNN = 27, image = True)

In [ ]:
df, text_predictions = get_neighbors(df, text_embeddings, KNN = 22, image = False)

In [ ]:
from cuml.feature_extraction.text import TfidfVectorizer

model = TfidfVectorizer(stop_words=None, binary=True, max_features=25000)
text_embeddings2 = model.fit_transform(df_cu.title).toarray()
print('text embeddings shape',text_embeddings2.shape)

In [ ]:
preds = []
CHUNK = 1024*4

print('Finding similar titles...')
CTS = len(df_cu)//CHUNK
if len(df_cu)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(df_cu))
    print('chunk',a,'to',b)
    
    # COSINE SIMILARITY DISTANCE
    # cts = np.dot( text_embeddings, text_embeddings[a:b].T).T
    cts = cupy.matmul(text_embeddings2, text_embeddings2[a:b].T).T
    
    for k in range(b-a):
        IDX = np.where(cts[k,]>0.74)[0] #0.738
        #IDX = cupy.where(cts[k,]>0.75)[0] # Orig
        o = df_cu.iloc[cupy.asnumpy(IDX)].posting_id.to_pandas().values
        preds.append(o)
        
del model, text_embeddings2

In [ ]:
df_cu['oof_text'] = preds

Images

In [ ]:
import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
class CFG:
    
    img_size = 512
    batch_size = 12
    seed = 2020
    
    device = 'cuda'
    classes = 11014
    
    model_name = 'eca_nfnet_l0'
    model_path = '../input/shopee-pytorch-models/arcface_512x512_nfnet_l0 (mish).pt'
    
    scale = 30 
    margin = 0.5

In [ ]:
def read_dataset():
    df = pd.read_csv('../input/shopee-product-matching/test.csv')
    df_cu = cudf.DataFrame(df)
    image_paths = '../input/shopee-product-matching/test_images/' + df['image']
    return df, df_cu, image_paths

def read_datasetTrain():
    df = pd.read_csv('../input/shopee-product-matching/train.csv')
    df_cu = cudf.DataFrame(df)
    image_paths = '../input/shopee-product-matching/train_images/' + df['image']
    return df, df_cu, image_paths

In [ ]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(CFG.seed)

In [ ]:
def get_image_predictions(df, embeddings,threshold = 0.0):
    
    if len(df) > 3:
        KNN = 50
    else : 
        KNN = 3
    
    model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    predictions = []
    for k in tqdm(range(embeddings.shape[0])):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = df['posting_id'].iloc[ids].values
        predictions.append(posting_ids)
        
    del model, distances, indices
    gc.collect()
    return predictions

In [ ]:
def get_test_transforms():

    return A.Compose(
        [
            A.Resize(CFG.img_size,CFG.img_size,always_apply=True),
            A.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

In [ ]:
class ShopeeDataset(Dataset):
    def __init__(self, image_paths, transforms=None):

        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
    
        return image,torch.tensor(1)

In [ ]:
class ArcMarginProduct_Image(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct_Image, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin
        
    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale
        
        return output

In [ ]:
class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes,
        model_name = CFG.model_name,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))
        
        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'efficientnet_b3':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'tf_efficientnet_b5_ns':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
            
        elif model_name == 'eca_nfnet_l0':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct_Image(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)
        
    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x

In [ ]:
class Mish_func(torch.autograd.Function):
    
    """from: https://github.com/tyunist/memory_efficient_mish_swish/blob/master/mish.py"""
    
    @staticmethod
    def forward(ctx, i):
        result = i * torch.tanh(F.softplus(i))
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
  
        v = 1. + i.exp()
        h = v.log() 
        grad_gh = 1./h.cosh().pow_(2)
        
        # Note that grad_hv * grad_vx = sigmoid(x)
        #grad_hv = 1./v  
        #grad_vx = i.exp()
        
        grad_hx = i.sigmoid()

        grad_gx = grad_gh *  grad_hx #grad_hv * grad_vx 
        
        grad_f =  torch.tanh(F.softplus(i)) + i * grad_gx 
        
        return grad_output * grad_f 
    
class Mish(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        pass
    def forward(self, input_tensor):
        return Mish_func.apply(input_tensor)
    
    
def replace_activations(model, existing_layer, new_layer):
    
    """A function for replacing existing activation layers"""
    
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            model._modules[name] = replace_activations(module, existing_layer, new_layer)

        if type(module) == existing_layer:
            layer_old = module
            layer_new = new_layer
            model._modules[name] = layer_new
    return model

In [ ]:
def get_image_embeddings1(image_paths, model_name = CFG.model_name):
    embeds = []
    
    model = ShopeeModel(model_name = model_name)
    model.eval()
    
    if model_name == 'eca_nfnet_l0':
        model = replace_activations(model, torch.nn.SiLU, Mish())

    model.load_state_dict(torch.load(CFG.model_path))
    model = model.to(CFG.device)
    

    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    
    del model
    
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

In [ ]:
df_image,df_image_cu,image_paths = read_dataset()
#df_image,df_image_cu,image_paths = read_datasetTrain()
df_image.head()

In [ ]:
def findDifference(f1, f2):
    return np.linalg.norm(f1-f2)
 
def cosine_distance(a, b):
    if a.shape != b.shape:
        raise RuntimeError("array {} shape not match {}".format(a.shape, b.shape))
    if a.ndim==1:
        a_norm = np.linalg.norm(a)
        b_norm = np.linalg.norm(b)
    elif a.ndim==2:
        a_norm = np.linalg.norm(a, axis=1, keepdims=True)
        b_norm = np.linalg.norm(b, axis=1, keepdims=True)
    else:
        raise RuntimeError("array dimensions {} not right".format(a.ndim))
    similiarity = np.dot(a, b.T)/(a_norm * b_norm) 
    dist = 1. - similiarity
    return dist

In [ ]:
def breakPostfixQuants(ttl,psfx):
    ttlB = []
    qTpls = []
    for wrd in ttl:
        found = False
        for fx in psfx:
            if wrd[-len(fx):]==fx:
                n = len(wrd)-len(fx)
                if n > 0:
                    prv = wrd[:n]
                    if prv.isnumeric():
                        found = True
                        ttlB.append(prv)
                        ttlB.append(fx)
        if not found:
            ttlB.append(wrd)
            
    psfx = set(psfx)
    for i,wrd in enumerate(ttlB):
        if i>0:
            if wrd in psfx:
                prv = ttlB[i-1]
                if prv.isnumeric():
                    qTpls.append( (prv,wrd) )
                    
    for tpl in qTpls:
        ttlB.remove(tpl[0])
        ttlB.remove(tpl[1])
            
    return (ttlB,qTpls)

In [ ]:
def preprocessTitle(ttl):
    tl = ttl.split(' ')
    #tl = [wd for wd in tl if wd not in allSW and len(wd)>0]
    tl = [wd for wd in tl if len(wd)>0]
    psfx = ['ML','LTR','KG','GM','GR','GRM','GRAM','PC','PCS','L','LBS','PAKET','PACKET', \
            'X','W','V','IN','G','CM','MM','M','OZ','GB','INCHES','FT','PIECES','SHEETS',\
           'PACK','PACKS']
    tl,qtpl = breakPostfixQuants(tl,psfx)
    #btl = set(zip(tl,tl[1:]))
    #return (tl,qtpl,btl)
    return (tl,qtpl)

In [ ]:
def quantMismatched(Q1,Q2):
    Q1 = set([q[0] for q in Q1 ])
    Q2 = set([q[0] for q in Q2 ])
    nf = False
    if len(Q1)>0 and len(Q2)>0:
        QL = Q1
        QS = Q2
        if len(QS)>len(QL):
            QS = Q1
            QL = Q2
        nf = False
        for q in QS:
            if q not in QL:
                nf = True
                break
    return nf

In [ ]:
pptDict = {}
for i in tqdm(range(df.shape[0])):
    ttl = df['title'].iloc[i]
    tl,qtpl = preprocessTitle(ttl)
    pid = df['posting_id'].iloc[i]
    pptDict[pid] = (tl,qtpl)

In [ ]:
image_embeddings1 = get_image_embeddings1(image_paths.values)
image_predictions1 = get_image_predictions(df_image, image_embeddings1, threshold = 0.314) #0.3

In [ ]:
# Function to combine predictions
def combine_predictions(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions'], row['oof_text'], row['oof_hash']])
    return ' '.join( np.unique(x) )

In [ ]:
# Function to combine predictions
def combine_predictions_oof(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions'], row['oof_text'], row['oof_hash'],row['oof_image']])
    return ' '.join( np.unique(x) )

In [ ]:
# Concatenate image predctions with text predictions
tmp = df.groupby('image_phash').posting_id.agg('unique').to_dict()
df['oof_hash'] = df.image_phash.map(tmp)
df['image_predictions'] = image_predictions
df['oof_text'] = df_cu['oof_text'].to_pandas().values
df['text_predictions'] = text_predictions
df['oof_image']=image_predictions1

In [ ]:
df.head(5)

In [ ]:
addPhaSure = True
addTitleSure = True
useJIThresh = True
JIThresh = 0.416 

for i in tqdm(range(df.shape[0])):
    pID = df['posting_id'].iloc[i]
    matchesTL = list(df['text_predictions'].iloc[i])
    matchesIL = list(df['image_predictions'].iloc[i])
    tl1 = pptDict[pID][0]
    lst2 = []
    for mtID in matchesTL:
        if mtID != pID and useJIThresh:
            tl2 = pptDict[mtID][0]
            aUbU = len( set(tl1).union(set(tl2)) )
            aIbU = len( set(tl1).intersection(set(tl2)) )
            Jid = aIbU / aUbU
            if Jid > JIThresh:
                lst2.append(mtID)
        else:
            lst2.append(mtID)
            
    if addTitleSure:
        qTitle = df['title'].iloc[i]
        titleLst = list(df[ df['title'] == qTitle  ].index)
        titleLst = [ df['posting_id'].iloc[tid] for tid in titleLst ]
        titleSet = set(titleLst)
        lst2Set = set(lst2)
        lst2Set2 = titleSet.union(lst2Set)
        lst2 = list(lst2Set2)
        
    lst2 = np.array(lst2).astype(object)
    df['text_predictions'].iloc[i] = lst2
    
    lst3 = []
    
    for mtID in matchesIL:
        lst3.append(mtID)
        
    if addPhaSure:
        qHash = df['image_phash'].iloc[i]
        phLst = list( df[ df['image_phash'] == qHash ].index)
        phLst = [ df['posting_id'].iloc[tid] for tid in phLst ]
        phSet = set(phLst)
        lst3Set = set(lst3)
        lst3Set2 = phSet.union(lst3Set)
        lst3 = list(lst3Set2)
        
    lst3 = np.array(lst3).astype(object)
    df['image_predictions'].iloc[i] = lst3
        

In [ ]:
df['matches'] = df.apply(combine_predictions_oof, axis = 1)

In [ ]:
sub_df = df[['posting_id', 'matches']] 

In [ ]:
pidToIndx = {}
for i in tqdm(range(df.shape[0])):
    pID = df['posting_id'].iloc[i]
    pidToIndx[pID] = i

In [ ]:

delCnt = 0
doRec = False
featsDict = {'TL1':[],'TL2':[], 'JI':[],'SE':[],'isDup':[]}
testMode = False
for i in tqdm(range(sub_df.shape[0])):
    pID = sub_df['posting_id'].iloc[i]
    idIndx = pidToIndx[pID]
    matchesStr = sub_df['matches'].iloc[i]
    matchesLst = matchesStr.split(' ')
    matchesLst2 = [] 
    if len(matchesLst)>1:
        #print(pID)
        #print(matchesStr)
        #print(matchesLst)
        
        Q1 = pptDict[pID][1]
        tl1 = pptDict[pID][0]
        
        for mtID in matchesLst:
            if mtID != pID:
                Q2 = pptDict[mtID][1]
                
                tl2 = pptDict[mtID][0]
                
                qMM = quantMismatched(Q1,Q2)
                if qMM:
                    qMM=1
                else:
                    qMM=0
                    
                ###############
                #f1 = image_embeddings[idIndx]
                #idIndx2 = pidToIndx[mtID]
                #f2 = image_embeddings[idIndx2]
                #se = cosine_distance(f1,f2)
                
                ###############

                if qMM==0: 
                    matchesLst2.append(mtID)
                    

                    if testMode:
                        idIndx2 = pidToIndx[mtID]
                        lbl1 = df['label_group'][idIndx]
                        lbl2 = df['label_group'][idIndx2]
                        duplicTruth = lbl1==lbl2
                        if duplicTruth == False:
                            print(tl1,tl2)
                    
                else:
                    delCnt += 1
                    #print(delCnt,' deleted')
                    
                
                    
                if doRec:
                    aUbU = len( set(tl1).union(set(tl2)) )
                    aIbU = len( set(tl1).intersection(set(tl2)) )
                    Jid = aIbU / aUbU
                    f1 = image_embeddings[idIndx]
                    idIndx2 = pidToIndx[mtID]
                    f2 = image_embeddings[idIndx2]
                    se = cosine_distance(f1,f2)
                    lbl1 = df['label_group'][idIndx]
                    lbl2 = df['label_group'][idIndx2]
                    duplicTruth = lbl1==lbl2
                    featsDict['JI'].append(Jid)
                    featsDict['TL1'].append(tl1)
                    featsDict['TL2'].append(tl2)
                    featsDict['SE'].append(se)
                    if duplicTruth:
                        featsDict['isDup'].append(1)
                    else:
                        featsDict['isDup'].append(0)
                    
            else:
                matchesLst2.append(mtID)
                
        mS2 = ''
        for m2 in matchesLst2:
            mS2 += m2+' '
        mS2 = mS2[:-1]
        sub_df['matches'].iloc[i] = mS2
                
        #print('*****')

In [ ]:
sub_df.to_csv('submission.csv', index = False)